<a href="https://colab.research.google.com/github/sohansputhran/cats-and-dogs-classifier/blob/master/Dogs_%26_Cats_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [0]:
! kaggle datasets download 'chetankv/dogs-cats-images'

In [0]:
!mkdir dogs-cats-images
! unzip dogs-cats-images.zip -d dogs-cats-images

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [0]:
data_dir = "/content/dogs-cats-images/dataset"
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/training_set', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test_set', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [0]:
model = models.densenet121(pretrained=True)
model

In [0]:
from google.colab.patches import cv2_imshow
import cv2 

img = cv2.imread("/content/dogs-cats-images/dataset/training_set/dogs/dog.1.jpg")
cv2_imshow(img)

In [0]:
import time
import numpy as np

In [0]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

In [0]:
# initialize tracker for minimum validation loss
test_loss_min = np.Inf # set initial "min" to infinity

epochs = 1
steps = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        train_loss = 0
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            test_loss = test_loss/len(testloader)        
            print(f"Epoch {epoch+1}/{steps}.. "
                  f"Train loss: {train_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss:.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            model.train()
            # save model if test loss has decreased
            if test_loss <= test_loss_min:
                print('Test loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                  test_loss_min,
                  test_loss))
                torch.save(model.state_dict(), 'model.pt')
                test_loss_min = test_loss

In [0]:
model.load_state_dict(torch.load('/content/model.pt'))
model.eval()

In [0]:
from PIL import Image
from torchvision.transforms import ToTensor
from torch.autograd import Variable


def image_loader(image_name):
    """load image, returns cuda tensor"""
    loader = transforms.Compose([transforms.ToTensor()])
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU
  
def classifier(image):
  data = torch.exp(model(image))
  data = data.cpu().data.numpy()[0]
  print(data)
  if data[0] > data[1]: print('CAT')
  else: print("DOG") 

In [0]:
image = image_loader("/content/dogs-cats-images/dataset/training_set/dogs/dog.1.jpg")
classifier(image)
# cla = torch.exp(model(image))
# cla

In [0]:
image = image_loader("/content/dogs-cats-images/dataset/test_set/cats/cat.4001.jpg")
classifier(image)